In [4]:
# Copyright (c) Recommenders contributors.
# Licensed under the MIT License.

import random
import numpy as np
import pandas as pd
import scipy.sparse as sp



class ImplicitCF(object):
    """Data processing class for GCN models which use implicit feedback.

    Initialize train and test set, create normalized adjacency matrix and sample data for training epochs.

    """

    def __init__(
        self,
        train,
        test=None,
        adj_dir=None,
        col_user='userID',
        col_item='itemID',
        col_rating='rating',
        col_prediction='prediction',
        seed=None,
    ):
        """Constructor

        Args:
            adj_dir (str): Directory to save / load adjacency matrices. If it is None, adjacency
                matrices will be created and will not be saved.
            train (pandas.DataFrame): Training data with at least columns (col_user, col_item, col_rating).
            test (pandas.DataFrame): Test data with at least columns (col_user, col_item, col_rating).
                test can be None, if so, we only process the training data.
            col_user (str): User column name.
            col_item (str): Item column name.
            col_rating (str): Rating column name.
            seed (int): Seed.

        """
        self.user_idx = None
        self.item_idx = None
        self.adj_dir = adj_dir
        self.col_user = col_user
        self.col_item = col_item
        self.col_rating = col_rating
        self.col_prediction = col_prediction
        self.train, self.test = self._data_processing(train, test)
        self._init_train_data()

        random.seed(seed)

    def _data_processing(self, train, test):
        """Process the dataset to reindex userID and itemID and only keep records with ratings greater than 0.

        Args:
            train (pandas.DataFrame): Training data with at least columns (col_user, col_item, col_rating).
            test (pandas.DataFrame): Test data with at least columns (col_user, col_item, col_rating).
                test can be None, if so, we only process the training data.

        Returns:
            list: train and test pandas.DataFrame Dataset, which have been reindexed and filtered.

        """
        df = (
            train
            if test is None
            else pd.concat([train, test], axis=0, ignore_index=True)
        )

        if self.user_idx is None:
            user_idx = df[[self.col_user]].drop_duplicates().reindex()
            user_idx[self.col_user + "_idx"] = np.arange(len(user_idx))
            self.n_users = len(user_idx)
            self.n_users_in_train = train[self.col_user].nunique()
            self.user_idx = user_idx

            self.user2id = dict(
                zip(user_idx[self.col_user], user_idx[self.col_user + "_idx"])
            )
            self.id2user = dict(
                zip(user_idx[self.col_user + "_idx"], user_idx[self.col_user])
            )

        if self.item_idx is None:
            item_idx = df[[self.col_item]].drop_duplicates()
            item_idx[self.col_item + "_idx"] = np.arange(len(item_idx))
            self.n_items = len(item_idx)
            self.item_idx = item_idx

            self.item2id = dict(
                zip(item_idx[self.col_item], item_idx[self.col_item + "_idx"])
            )
            self.id2item = dict(
                zip(item_idx[self.col_item + "_idx"], item_idx[self.col_item])
            )

        return self._reindex(train), self._reindex(test)

    def _reindex(self, df):
        """Process the dataset to reindex userID and itemID and only keep records with ratings greater than 0.

        Args:
            df (pandas.DataFrame): dataframe with at least columns (col_user, col_item, col_rating).

        Returns:
            list: train and test pandas.DataFrame Dataset, which have been reindexed and filtered.

        """

        if df is None:
            return None

        df = pd.merge(df, self.user_idx, on=self.col_user, how="left")
        df = pd.merge(df, self.item_idx, on=self.col_item, how="left")

        df = df[df[self.col_rating] > 0]

        df_reindex = df[
            [self.col_user + "_idx", self.col_item + "_idx", self.col_rating]
        ]
        df_reindex.columns = [self.col_user, self.col_item, self.col_rating]
        print(df_reindex)
        return df_reindex

    def _init_train_data(self):
        """Record items interated with each user in a dataframe self.interact_status, and create adjacency
        matrix self.R.

        """
        self.interact_status = (
            self.train.groupby(self.col_user)[self.col_item]
            .apply(set)
            .reset_index()
            .rename(columns={self.col_item: self.col_item + "_interacted"})
        )
        self.R = sp.dok_matrix((self.n_users, self.n_items), dtype=np.float32)
        self.R[self.train[self.col_user], self.train[self.col_item]] = 1.0

    def get_norm_adj_mat(self):
        """Load normalized adjacency matrix if it exists, otherwise create (and save) it.

        Returns:
            scipy.sparse.csr_matrix: Normalized adjacency matrix.

        """
        try:
            if self.adj_dir is None:
                raise FileNotFoundError
            norm_adj_mat = sp.load_npz(self.adj_dir + "/norm_adj_mat.npz")
            print("Already load norm adj matrix.")

        except FileNotFoundError:
            norm_adj_mat = self.create_norm_adj_mat()
            if self.adj_dir is not None:
                sp.save_npz(self.adj_dir + "/norm_adj_mat.npz", norm_adj_mat)
        return norm_adj_mat

    def create_norm_adj_mat(self):
        """Create normalized adjacency matrix.

        Returns:
            scipy.sparse.csr_matrix: Normalized adjacency matrix.

        """
        adj_mat = sp.dok_matrix(
            (self.n_users + self.n_items, self.n_users + self.n_items), dtype=np.float32
        )
        adj_mat = adj_mat.tolil()
        R = self.R.tolil()

        adj_mat[: self.n_users, self.n_users :] = R
        adj_mat[self.n_users :, : self.n_users] = R.T
        adj_mat = adj_mat.todok()
        print("Already create adjacency matrix.")

        rowsum = np.array(adj_mat.sum(1))
        d_inv = np.power(rowsum + 1e-9, -0.5).flatten()
        d_inv[np.isinf(d_inv)] = 0.0
        d_mat_inv = sp.diags(d_inv)
        norm_adj_mat = d_mat_inv.dot(adj_mat)
        norm_adj_mat = norm_adj_mat.dot(d_mat_inv)
        print("Already normalize adjacency matrix.")

        return norm_adj_mat.tocsr()

    def train_loader(self, batch_size):
        """Sample train data every batch. One positive item and one negative item sampled for each user.

        Args:
            batch_size (int): Batch size of users.

        Returns:
            numpy.ndarray, numpy.ndarray, numpy.ndarray:
            - Sampled users.
            - Sampled positive items.
            - Sampled negative items.
        """

        def sample_neg(x):
            if len(x) >= self.n_items:
                raise ValueError("A user has voted in every item. Can't find a negative sample.")
            while True:
                neg_id = random.randint(0, self.n_items - 1)
                if neg_id not in x:
                    return neg_id

        indices = range(self.n_users_in_train)
        if self.n_users < batch_size:
            users = [random.choice(indices) for _ in range(batch_size)]
        else:
            users = random.sample(indices, batch_size)

        interact = self.interact_status.iloc[users]
        pos_items = interact[self.col_item + "_interacted"].apply(
            lambda x: random.choice(list(x))
        )
        neg_items = interact[self.col_item + "_interacted"].apply(
            lambda x: sample_neg(x)
        )

        return np.array(users), np.array(pos_items), np.array(neg_items)

In [1]:
import numpy as np
import pandas as pd
import splitter
import movielens
import graphLoader 

In [2]:
split = splitter.random_split(movielens.load_pandas_df("100K"))

/home/vinmike/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [5]:
data = ImplicitCF(train=split[0], test=split[1])
# print(data.test)

       userID  itemID  rating
0           0       0     3.0
1           1       1     4.0
2           2       2     4.0
3           3       3     4.0
4           4       4     3.0
...       ...     ...     ...
79994     166      24     3.0
79995     751     212     4.0
79996     121     161     3.0
79997     139     532     5.0
79998     142     156     4.0

[79999 rows x 3 columns]
       userID  itemID  rating
0         296     281     5.0
1         627     352     3.0
2         255     294     5.0
3         106     451     5.0
4         725     492     1.0
...       ...     ...     ...
19995     222      73     3.0
19996     300     132     3.0
19997       2     856     5.0
19998      75     179     5.0
19999     510      66     2.0

[20000 rows x 3 columns]


In [6]:
data = graphLoader.ImplicitCF(train=split[0], test=split[1])
# print(data.test)

IndexError: index (943) out of range

In [6]:
print(data.get_norm_adj_mat())

Already create adjacency matrix.
Already normalize adjacency matrix.
  (0, 943)	0.0314970389008522
  (0, 987)	0.012123215943574905
  (0, 1009)	0.013263952918350697
  (0, 1035)	0.017323972657322884
  (0, 1052)	0.01200038380920887
  (0, 1084)	0.010631080716848373
  (0, 1089)	0.01918824017047882
  (0, 1094)	0.012543190270662308
  (0, 1154)	0.01277023646980524
  (0, 1155)	0.01585903763771057
  (0, 1161)	0.0144942756742239
  (0, 1200)	0.0144942756742239
  (0, 1202)	0.016835875809192657
  (0, 1223)	0.013856849633157253
  (0, 1229)	0.017039954662322998
  (0, 1277)	0.015915175899863243
  (0, 1291)	0.010531567968428135
  (0, 1397)	0.018101215362548828
  (0, 1498)	0.029880713671445847
  (0, 1499)	0.016386838629841805
  (0, 1503)	0.012828949838876724
  (0, 1560)	0.015803487971425056
  (0, 1576)	0.0306569654494524
  (0, 1699)	0.024397501721978188
  (0, 1879)	0.029160592705011368
  :	:
  (2571, 118)	0.04019339382648468
  (2572, 118)	0.04019339382648468
  (2573, 300)	0.06468462198972702
  (2574, 403

In [16]:
data.train_loader(2)


(array([764,  52]), array([868, 304]), array([615, 956]))